In [ ]:
# %load metamaps.py
"""Provides methods for loading/saving metamaps"""
import struct
from enum import IntEnum
from keras.utils import Sequence
import numpy as np
import os
import random

METAMAP_FILE_VERSION = 0x100

class TileType(IntEnum):
    """Tile types in a metamap"""
    UNREACHABLE = 0
    EMPTY = 1
    WALL = 2
    PUSHWALL = 3
    DOOR = 4

class EncodingDim(IntEnum):
    """Dimensions for the one-hot encoding of a metamap"""
    PLAYABLE = 0
    SOLID = 1
    PASSAGE = 2

TileTypeToEncodingDim = {
    TileType.UNREACHABLE: EncodingDim.SOLID,
    TileType.EMPTY: EncodingDim.PLAYABLE,
    TileType.WALL: EncodingDim.SOLID,
    TileType.PUSHWALL: EncodingDim.PASSAGE,
    TileType.DOOR: EncodingDim.PASSAGE,
    }

class MetaMapsSequence(Sequence):
    """A sequence of real metamaps from a directory and randomly generated ones"""

    def __init__(self, maps_dir, batch_size):
        self.maps_dir = maps_dir
        self.batch_size = batch_size
        self.map_files = os.listdir(maps_dir)
        NUM_MAPS = len(self.map_files)
        real_maps = [(index, True) for index in range(NUM_MAPS)]
        fake_maps = [(index + NUM_MAPS, False) for index in range(NUM_MAPS)]
        map_order = real_maps + fake_maps
        np.random.shuffle(map_order)
        self.map_order = map_order

    def __len__(self):
        return int(len(self.map_order) / self.batch_size)

    def __getitem__(self, idx):
        map_batch = np.zeros((self.batch_size, 64, 64, len(EncodingDim)))
        label_batch = np.zeros((self.batch_size))

        for i in range(self.batch_size):
            (index, real_map) = self.map_order[idx * self.batch_size + i]
            if real_map:
                label_batch[i] = 1
                map_batch[i:] = load_metamap(
                    os.path.join(self.maps_dir, self.map_files[index]))
            else:
                map_batch[i:] = generate_random_map()

        return map_batch, label_batch


def generate_random_map():
    """Generate a random map"""
    width = 64
    height = 64
    size = width * height

    junk_map = np.zeros([size, len(EncodingDim)])

    for i in range(size):
        tile_type = random.randint(0, len(EncodingDim) - 1)
        junk_map[i, tile_type] = 1

    junk_map.shape = (width, height, len(EncodingDim))

    return junk_map


def load_all_metamaps(dirname):
    """Loads all the metamaps in the given directory, returning a giant numpy array"""
    map_names = os.listdir(dirname)
    all_maps = np.zeros((len(map_names), 64, 64, len(EncodingDim)))

    for index, map_name in enumerate(map_names):
        load_metamap_into(os.path.join(dirname, map_name), all_maps, index)

    return all_maps


def load_metamap_into(filename, all_maps, index):
    """Loads a metamap from a file into a numpy array of shape (width, height, 3)"""
    with open(filename, "rb") as fin:
        version = struct.unpack('Q', fin.read(8))[0]

        if version != METAMAP_FILE_VERSION:
            raise ValueError("Unsupported metamap version")

        width = struct.unpack('i', fin.read(4))[0]
        height = struct.unpack('i', fin.read(4))[0]

        raw_map = np.fromfile(fin, dtype=np.uint8)
        raw_map.shape = (width, height)

        for y in range(height):
            for x in range(width):
                tile_type = TileType(raw_map[y, x])
                all_maps[index, y, x, TileTypeToEncodingDim[tile_type]] = 1


def load_metamap(filename):
    """Loads a metamap from a file into a numpy array of shape (width, height, 3)"""
    with open(filename, "rb") as fin:
        version = struct.unpack('Q', fin.read(8))[0]

        if version != METAMAP_FILE_VERSION:
            raise ValueError("Unsupported metamap version")

        width = struct.unpack('i', fin.read(4))[0]
        height = struct.unpack('i', fin.read(4))[0]
        size = width * height

        raw_map = np.fromfile(fin, dtype=np.uint8)
        one_hot = np.zeros([size, len(EncodingDim)])

        for i in range(size):
            tile_type = TileType(raw_map[i])
            one_hot[i, TileTypeToEncodingDim[tile_type]] = 1

        one_hot.shape = (width, height, len(EncodingDim))

        return one_hot


def save_metamap(metamap, filename):
    """Saves a metamap to a file"""
    with open(filename, "wb") as fout:
        fout.write(struct.pack('Q', METAMAP_FILE_VERSION))

        width = metamap.shape[0]
        height = metamap.shape[1]

        fout.write(struct.pack('i', width))
        fout.write(struct.pack('i', height))
        for y in range(height):
            for x in range(width):
                tile_type = TileType.WALL
                if metamap[y, x, EncodingDim.PLAYABLE] == 1:
                    tile_type = TileType.EMPTY
                elif metamap[y, x, EncodingDim.SOLID] == 1:
                    tile_type = TileType.WALL
                elif metamap[y, x, EncodingDim.PASSAGE] == 1:
                    tile_type = TileType.DOOR

                fout.write(struct.pack('b', tile_type))
    return

# Load all maps

In [ ]:
import time

t0 = time.perf_counter()
all_maps = load_all_metamaps("metamaps")
t1 = time.perf_counter()
time_spent_loading = t1 - t0

In [ ]:
time_spent_str = time.strftime("%H:%M:%S", time.gmtime(time_spent_loading))
print(f"Loaded all maps in {time_spent_str}")

In [ ]:
print(f"all_maps.shape = {all_maps.shape}")

In [ ]:
batch_size = 20

all_maps_len = all_maps.shape[0]
all_maps_len = all_maps_len - (all_maps_len % batch_size)

print(all_maps_len)

In [ ]:
batches = int(all_maps_len/batch_size)
test_size = int((1/7)*batches) * batch_size
print(test_size)

In [ ]:
train_size = all_maps_len - test_size

In [ ]:
train_maps = all_maps[:train_size, :, :, :]
test_maps = all_maps[train_size:(train_size+test_size), :, :, :]
print("Train shape: ",train_maps.shape)
print("Test shape:  ",test_maps.shape)

# Setup Keras

In [ ]:
import keras
from keras import layers
from keras import backend as K
from keras.models import Model
import numpy as np

In [ ]:
map_shape = (64, 64, len(EncodingDim))

# Encoder

In [ ]:
FILTERS = 64

input_map = keras.Input(shape=map_shape)

# keras.layers.Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', data_format=None, dilation_rate=(1, 1), activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)

#layer 1
x = layers.SeparableConv2D(FILTERS, 3, 
                  padding='same', 
                  activation='relu')(input_map)
x = layers.MaxPooling2D((2,2))(x)

#layer 2
x = layers.SeparableConv2D(FILTERS, 3,
                  padding='same', 
                  activation='relu')(x)

x = layers.MaxPooling2D((2,2))(x)

#layer 3
x = layers.SeparableConv2D(FILTERS, 3,
                  padding='same', 
                  activation='relu')(x)
encoded = layers.MaxPooling2D((2,2))(x)

# Decoder

In [ ]:
#layer 1
x = layers.SeparableConv2D(FILTERS, 3,
                           padding='same', activation='relu')(encoded)
x = layers.UpSampling2D()(x)

#layer 2
x = layers.SeparableConv2D(FILTERS, 3,
                           padding='same', activation='relu')(x)
x = layers.UpSampling2D()(x)

#layer 3
x = layers.SeparableConv2D(FILTERS, 3,
                  padding='same', activation='relu')(x)
x = layers.UpSampling2D()(x)

#final (Should this be relu or sigmoid?  I think not sigmoid because size is len(EncodingDim))
decoded = layers.SeparableConv2D(len(EncodingDim), 3,
                  padding='same', activation='sigmoid')(x)

# Create Model (why is loss=none?)

In [ ]:
autoencoder = Model(input_map, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.summary()

# Training!

In [ ]:
history = autoencoder.fit(
    x=train_maps, y=train_maps,
    shuffle=False,
    epochs=5,
    batch_size=batch_size,
    validation_data=(test_maps, test_maps))

# Show Output (FIX to work with new auto-encoder)

In [ ]:
import matplotlib.pyplot as plt

decoded_maps = autoencoder.predict(test_maps)

n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i)
    plt.imshow(test_maps[i].reshape(map_shape))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + n)
    plt.imshow(decoded_maps[i].reshape(map_shape))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

# Plot training and validation loss

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Plot training and validation accuracy

In [ ]:
plt.clf()   # clear figure
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()